In [73]:
import cv2 as cv 
import numpy as np
import os
from tensorflow import keras

In [46]:
class Line:
    """
    Store a line based on the two points.
    """
    def __init__(self, point_1, point_2):
        self.point_1 = point_1
        self.point_2 = point_2
        

class Point:
    def __init__(self, x, y):
        self.x = np.int32(np.round(x))
        self.y = np.int32(np.round(y))
    
    def get_point_as_tuple(self):
        return (self.x, self.y)
    
        
class Patch:
    """
    Store information about each item (where the mark should be found) in the table. 
    """
    def __init__(self, image_patch, x_min, y_min, x_max, y_max, line_idx, column_idx):
        self.image_patch = image_patch
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        self.line_idx = line_idx
        self.column_idx = column_idx
        self.domino_value = 0 # 0 meaning it does not contain any domino
    
    def set_domino_value(self, has_x: int):
        assert any([has_x == i for i in range(7)])
        self.has_x = has_x

In [47]:
def show_image(image, window_name='image', timeout=0):
    """
    :param timeout. How many seconds to wait untill it close the window.
    """
    cv.imshow(window_name, cv.resize(image, None, fx=0.6, fy=0.6))
    cv.waitKey(timeout)
    cv.destroyAllWindows()

def draw_lines(image, lines: list[Line], timeout: int = 0, color: tuple = (0, 0, 255),
               return_drawing: bool = False, window_name: str = 'window'):
    """
    Plots the lines into the image.
    :param image.
    :param lines.
    :param timeout. How many seconds to wait untill it close the window.
    :param color. The color used to draw the lines
    :param return_drawing. Use it if you want the drawing to be return instead of displayed.
    :return None if return_drawing is False, otherwise returns the drawing.
    """
    drawing = image.copy()
    if drawing.ndim == 2:
        drawing = cv.cvtColor(drawing, cv.COLOR_GRAY2BGR)
    for line in lines: 
        cv.line(drawing, line.point_1.get_point_as_tuple(), line.point_2.get_point_as_tuple(), color, 2, cv.LINE_AA)
        
    if return_drawing:
        return drawing
    else:
        show_image(drawing, window_name=window_name, timeout=timeout)

In [48]:
def get_patches(lines: list[Line], columns: list[Line], image, show_patches: bool = False) -> list[Patch]:
    """
    It cuts out each box from the table defined by the lines and columns.
    :param lines. The lines that difine the table.
    :param columns. The columns that difine the table.
    :param image. The image containing the table.
    :param show_patches. Determine if the patches will be drawn on the image or not.
    :return : A list with all boxes in the table.
    """
    
    def crop_patch(image_, x_min, y_min, x_max, y_max):
        """
        Crops the bounding box represented by the coordinates.
        """
        return image_[y_min: y_max, x_min: x_max].copy()
    
    def draw_patch(image_, patch: Patch, color: tuple = (255, 0, 255)):
        """
        Draw the bounding box corresponding to the patch on the image.
        """
        cv.rectangle(image_, (patch.x_min, patch.y_min), (patch.x_max, patch.y_max), color=color, thickness=5)
    
    assert image.ndim == 2
    if show_patches: 
        image_color = np.dstack((image, image, image))
  
    lines.sort(key=lambda line: line.point_1.y)
    columns.sort(key=lambda column: column.point_1.x)
    patches = []
    step = 0
    for line_idx in range(len(lines) - 1):
        for col_idx in range(len(columns) - 1):
            current_line = lines[line_idx]
            next_line = lines[line_idx + 1] 
            
            y_min = current_line.point_1.y + step
            y_max = next_line.point_1.y - step
            
            current_col = columns[col_idx]
            next_col = columns[col_idx + 1]
            x_min = current_col.point_1.x + step 
            x_max = next_col.point_1.x - step
            
            patch = Patch(image_patch=crop_patch(image,  x_min, y_min, x_max, y_max),
                          x_min=x_min, y_min=y_min, x_max=x_max, y_max=y_max,
                          line_idx=line_idx, column_idx=col_idx)
            
            if show_patches:
                draw_patch(image_color, patch)
            
            patches.append(patch)
            
    if show_patches:
        show_image(image_color, window_name='patches', timeout=0)
    return patches

In [49]:
def find_patches_template(image, template, show_intermediate_results: bool = False):
    """
    This function finds the 'X' in the entire image using the following steps: 
    1. transforming the query image and the template to grayscale
    2. getting the edges for both images using Canny edge detector with threshold1=100 and threshold2=150
    3. obtaining the displacement vector between the query image and the template image
    4. allign the query image to the template
    5. obtain the first left/right corner of the first table and the second table
    6. getting the vertical/horizontal lines based on the table dimension and the number of vertical/horizontal lines based
    7. cut out the paches (the boxes) from the two tables based on the horizontal and vertical lines 
    """
    image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    query_image_edges = cv.Canny(image_gray, 200, 250) #the params were found by trial and error
    template_image_edges = cv.Canny(template_gray, 200, 250)
    
    if show_intermediate_results: 
        show_image(query_image_edges, window_name='query_image_edges', timeout=0)
        show_image(template_image_edges, window_name='template_image_edges', timeout=0)
        
    alligned_image = image
    
    left_corner = Point(x=0, y=0)
    right_corner = Point(x=len(template), y=len(template))

    step_x = (right_corner.x - left_corner.x) / 15
    step_y = (right_corner.y - left_corner.y) / 15
    
    def get_vertical_and_horizontal_lines(left_corner: Point, step_x_, step_y_): 
        vertical_lines = []
        horizontal_lines = []
        
        for i in range(16):
            xmin = np.uint64(left_corner.x + i * step_x_)
            line = Line(Point(x = xmin, y = 0), Point(x = xmin, y = 5000)) 
            vertical_lines.append(line)
        for i in range(16):
            ymin = np.uint64(left_corner.y + i * step_y_)
            line = Line(Point(x = 0, y = ymin), Point(x = 5000, y = ymin))
            horizontal_lines.append(line)

        return vertical_lines, horizontal_lines
     
    vertical_lines, horizontal_lines = get_vertical_and_horizontal_lines(left_corner, step_x, step_y) 
    
    alligned_image_gray = cv.cvtColor(alligned_image, cv.COLOR_BGR2GRAY)
    patches = get_patches(horizontal_lines, vertical_lines, alligned_image_gray, 
                          show_patches=show_intermediate_results)
    
    return patches

In [50]:
class DominoClassifier:
    """
    Classifier that determines the value of the domino
    """
    def __init__(self):
        self.threshold = 245
    
    def classify(self, patch: Patch) -> int:
        """
        Receive a Patch and return 1 if there is an 'X' in the pacth and 0 otherwise.
        """ 
        if patch.image_patch.mean() > self.threshold:
            return 0
        else: 
            return 1
        
        
def classify_patches_with_domino_classifier(patches: list[Patch]) -> None:
    """
    Receive the patches and classify if the patch contains an 'X' or not.
    :param patches.
    :return None
    """
    domino_classifier = DominoClassifier()
    for patch in patches:
        patch.set_domino_value(domino_classifier.classify(patch))

In [71]:
score_to_domino_dict = {
    0: 1,
    1: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    6: 0,
    7: 2,
    8: 5,
    9: 3,
    10: 4,
    11: 6,
    12: 2,
    13: 2,
    14: 0,
    15: 3,
    16: 5,
    17: 4,
    18: 1,
    19: 6,
    20: 2,
    21: 4,
    22: 5,
    23: 5,
    24: 0,
    25: 6,
    26: 3,
    27: 4,
    28: 2,
    29: 0,
    30: 1,
    31: 5,
    32: 1,
    33: 3,
    34: 4,
    35: 4,
    36: 4,
    37: 5,
    38: 0,
    39: 6,
    40: 3,
    41: 5,
    42: 4,
    43: 1,
    44: 3,
    45: 2,
    46: 0,
    47: 0,
    48: 1,
    49: 1,
    50: 2,
    51: 3,
    52: 6,
    53: 3,
    54: 5,
    55: 2,
    56: 1,
    57: 0,
    58: 6,
    59: 6,
    60: 5,
    61: 2,
    62: 1,
    63: 2,
    64: 5,
    65: 0,
    66: 3,
    67: 3,
    68: 5,
    69: 0,
    70: 6,
    71: 1,
    72: 4,
    73: 0,
    74: 6,
    75: 3,
    76: 5,
    77: 1,
    78: 4,
    79: 2,
    80: 6,
    81: 2,
    82: 3,
    83: 1,
    84: 6,
    85: 5,
    86: 6,
    87: 2,
    88: 0,
    89: 4,
    90: 0,
    91: 1,
    92: 6,
    93: 4,
    94: 4,
    95: 1,
    96: 6,
    97: 6,
    98: 3,
    99: 0
}

In [92]:
images_path = 'aligned_train\\'
template = cv.imread(images_path + 'aligned_1_01.jpg')

img_filenames = [images_path + f for f in os.listdir(images_path) if f.endswith('.jpg')]
for img_filename in img_filenames:
    original_image = cv.imread(img_filename) 
    patches = find_patches_template(original_image, template)
    for idx, patch in enumerate(patches):
        img_patch = original_image[patch.x_min : patch.x_max, 
                                   patch.y_min : patch.y_max]
        cv.imwrite('patches\\' + img_filename.split('\\')[-1].split('.')[0] + '_patch_' + str(idx + 1) + '.jpg', img_patch)
    break